# Radiative equilibrium model

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [37]:
class REM:

  # INPUTS: total time of simulation, time step, mass at surface, pressure at surface, pressure step,
  #   beta (SW absorption), alpha (surf reflectance), initial atmos temp, and r=frac decrease in sw absorption
  # OUTPUTS: nothing
  # SIDE EFFECTS: initializes and defines constants and arrays
  def __init__(self, tot_time, del_t = 3600, m_sur=50_000/9.8, p_sur=100_000.0, del_p=5000.0, beta=0.0, alpha=0.3, T_i=50.0, r=0.5):
    # Time constants
    self.tot_time = tot_time # simulation time in seconds
    self.del_t = del_t # time step in seconds

    # Determine number of layers and levels
    self.N = int(p_sur/del_p) # get number of layers and levels

    # For levels:
    self.S_0 = 1363.0 # W/m^2 (Solar irradiance)
    self.alpha = alpha # surface albedo
    self.ps = 1.0*np.arange(0, p_sur, del_p) # Pressures

    # For layers
    self.Cps = 1005*np.ones_like(self.ps) # Specific heat for air (J/K/kg
    self.epsilons = np.ones_like(self.ps) # LW emissivities
    for n in range(self.N - 1):
      mid_p = (self.ps[n] + self.ps[n+1])/2
      if mid_p  > 80_000:
        self.epsilons[n] = 0.3
      elif (mid_p > 40_000) & (mid_p <= 80_000):
        self.epsilons[n] = 0.2
      else:
        self.epsilons[n] = 0.1
    self.betas = beta * np.ones_like(self.ps) # SW absorptivities
    for n in range(1, self.N):
        self.betas[n] = self.betas[n-1]*r
    self.betas[-1] = 0
    self.ms = m_sur*np.ones_like(self.ps) # Masses
    self.Ts = T_i*np.ones_like(self.ps) # Temperatures


  # INPUT: none (everything initialized before)
  # OUTPUT: array of temperatures by time step and then by pressure height
  def run_sim(self):
    sigma = 5.67e-8 # in W/m^2/K^4
    all_Ts = 1.0*np.zeros((int(self.tot_time/self.del_t)+1, self.N))
    all_Ts[0] = self.Ts

    # loop through times
    for t in np.arange(0, self.tot_time + self.del_t, self.del_t):

      # empty arrays to hold fluxes
      LW_downs = 1.0*np.zeros(self.N)
      LW_ups = 1.0*np.zeros_like(LW_downs)
      SW_downs = 1.0*np.zeros_like(LW_downs)

      # get upward LW flux
      LW_ups[-1] = sigma*(self.Ts[-1]**4)
      for n in range(self.N-2, -1, -1): # just above surface (K=18) to TOA (K=0)
        LW_ups[n] = (1-self.epsilons[n])*LW_ups[n+1] + self.epsilons[n]*sigma*(self.Ts[n]**4)

      # get downward LW flux
      LW_downs[0] = 0
      LW_downs[1] = self.epsilons[0]*sigma*(self.Ts[0]**4)
      for n in range(2, self.N): # K=2 to surface (K=19)
        LW_downs[n] = (1-self.epsilons[n-1])*LW_downs[n-1] + self.epsilons[n-1]*sigma*(self.Ts[n-1]**4)

      # get downward SW flux
      SW_downs[0] = self.S_0/4
      for n in range(1, self.N-1): # TOA to surface
          SW_downs[n] = (1-self.betas[n-1])*SW_downs[n-1]
      SW_downs[-1] = (1-self.alpha)*SW_downs[-2]

      # convert levels to layers
      Flw_up = 1.0*np.zeros_like(LW_downs)
      Flw_down = 1.0*np.zeros_like(LW_downs)
      Fsw = 1.0*np.zeros_like(LW_downs)

      for n in range(self.N - 1):
        Fsw[n] = SW_downs[n] - SW_downs[n+1]
        Flw_down[n] = LW_downs[n] - LW_downs[n+1]
      Fsw[-1] = 0 - SW_downs[-1]
      Flw_down[-1] = 0 - LW_downs[-1]

      Flw_up[-1] = 0 - LW_ups[-1]
      for n in range(self.N-2, -1, -1):
        Flw_up[n] = LW_ups[n+1] - LW_ups[n]


      # get flux convergence for each layer
      del_F = Flw_up + Flw_down + Fsw

      # update temperatures and add to summary array
      self.Ts = self.Ts + self.del_t*(del_F/(self.ms*self.Cps))
      all_Ts[int(t/self.del_t)] = self.Ts

    return all_Ts

In [35]:
six_months_sim = REM(tot_time=6*30*24*3600)

In [36]:
six_months_sim.ps

array([     0.,   5000.,  10000.,  15000.,  20000.,  25000.,  30000.,
        35000.,  40000.,  45000.,  50000.,  55000.,  60000.,  65000.,
        70000.,  75000.,  80000.,  85000.,  90000.,  95000., 100000.])

In [20]:
Ts = six_months_sim.run_sim()

<ipython-input-18-09d21dceac9e>:55: RuntimeWarning: overflow encountered in scalar power
  LW_ups[-1] = sigma*(self.Ts[-1]**4)
<ipython-input-18-09d21dceac9e>:57: RuntimeWarning: overflow encountered in scalar power
  LW_ups[n] = (1-self.epsilons[n])*LW_ups[n+1] + self.epsilons[n]*sigma*(self.Ts[n]**4)
<ipython-input-18-09d21dceac9e>:63: RuntimeWarning: overflow encountered in scalar power
  LW_downs[n] = (1-self.epsilons[n-1])*LW_downs[n-1] + self.epsilons[n-1]*sigma*(self.Ts[n-1]**4)
<ipython-input-18-09d21dceac9e>:78: RuntimeWarning: invalid value encountered in scalar subtract
  Flw_down[n] = LW_downs[n] - LW_downs[n+1]
<ipython-input-18-09d21dceac9e>:84: RuntimeWarning: invalid value encountered in scalar subtract
  Flw_up[n] = LW_ups[n+1] - LW_ups[n]


In [32]:
Ts[1900]

array([  50.63192462,   50.71208041,   50.80014448,   50.8970945 ,
         51.0040068 ,   51.12206832,   51.25258981,   51.39702035,
         53.50563756,   54.39976721,   55.5135215 ,   56.90246548,
         58.63587817,   60.80019418,   63.50328967,   66.87981773,
         86.10435674,  101.51971838,  238.74545007, -511.72339557])

## Part 1) No ozone so B=0

## Part 2) Ozone